In [ ]:
import qrcode
from pyzbar import pyzbar
from PIL import Image
import serial
import cv2
from time import time , ctime
from datetime import datetime,timezone

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred)

db = firestore.client()
costumer_ref = db.collection(u'costumer')
slot_ref = db.collection(u'slot')

In [ ]:
# #initialize qr object
# def SaveQrCode(data): 
#     qr = qrcode.QRCode(
#         version =1,
#         box_size =10,
#         border=6)


#     #add data to qr code
#     qr.add_data(data)
#     qr.make(fit=True)

#     #create an image of qr code
#     image = qr.make_image(fill_color="black", back_color= "white")

#     #save on the local machine
#     image.save("TGBqrcode.png")
#     print("QR code has been generated successfully!")

In [ ]:
# def ReadQrCode():
#     #load qr code imge
#     image = Image.open("TGBqrcode.png") 

#     qr_code = pyzbar.decode(image)[0]

#     #convert into string
#     data= qr_code.data.decode("utf-8")
#     type = qr_code.type
#     text = f"{type}-->, {data}"
#     balance = text.split(",")[2]
#     SaveQrCode("car1,"+str(int(balance)-10))
#     image.close()
   
#     print("New balance = R"+balance)
#     print("----")

In [ ]:
Price = 5 # will differ by how much a customer spend in the parking lot
slot_id = "tzUSNfZxiqnUyfJjuGi1"
global costumer

In [ ]:
conectionToArduino = serial.Serial('com5',9600) 
 
while (True) :
    if(conectionToArduino.inWaiting()>0):

        
        
        

        # camera section 
        loop = True # for controlling web cam
        cam = cv2.VideoCapture(0)
        scanner = cv2.QRCodeDetector()

        while loop:
            _,img = cam.read()
            identifier,_,_ = scanner.detectAndDecode(img)
            
            cv2.imshow("QRCODE",img)
            cv2.waitKey(500)
            if identifier :
                #update number of slots 
                slots = conectionToArduino.readline().decode()
                
                slot_ref.document(slot_id).update({u'slots':slots})
                # retrieve costumer information
               
                
                for doc in costumer_ref.stream():
                    if doc.to_dict()['identifier'] == int(identifier):
                        costumer = doc
                        break


                costumer_id,name, balance, status , dateIn  = costumer.id, costumer.to_dict()['name'],costumer.to_dict()['balance'],costumer.to_dict()['status'],costumer.to_dict()['dateIn']

               
                
                loop = False

                #the customer was inside
                if(status == 1):
                    timeSpent= abs(((datetime.now(timezone.utc)-doc.to_dict()['dateIn'])).total_seconds()/3600 )
                    print("time spent = "+str(timeSpent)+"seconds")
                    if timeSpent < 5:
                        balance = balance - Price
                    elif timeSpent >= 5 and timeSpent <10:
                        balance = balance -( Price*2)
                    else:
                        balance = balance - (Price*3)

                  
                    costumer_ref.document(costumer_id).update({u'balance':balance,u'status':0})
                    #ReadQrCode()

                #The customer is entering   
                else:
                    costumer_ref.document(costumer_id).update({u'status':1})

                    

                break 
            
        if cv2.waitKey(1) & 0xFF == 27:
            break
cv2.destroyAllWindows()
        
        
     